In [30]:
import torch
import pandas as pd
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from data_treatment import DataSet, DataAtts
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import glob

In [31]:
folder = widgets.Dropdown(
    options=glob.glob("models/heart*"),
    description='Folder:',
    # value="models/diabetes_escalonated",
    disabled=False,
)
display(folder)

Dropdown(description='Folder:', options=('models/heart-disease-binary-t1_TrainUpsampledPatient', 'models/heart…

In [32]:


folder_name = folder.value+"/generator*.pt"
model_widget = widgets.Dropdown(
    options=glob.glob(folder_name),
    description='Generator:',
    disabled=False,
)
display(model_widget)

Dropdown(description='Generator:', options=('models/heart-disease-binary-t5_TrainUpsampledPatient/generatorhea…

In [33]:
original_db_name = folder.value[7:]
# original_db_path = "original_data/" + original_db_name + ".csv"
original_db_path = "grg_data/" + original_db_name + ".csv"
original_db = pd.read_csv(original_db_path)
original_db_size=original_db.shape[0]

In [34]:
try:
    checkpoint= torch.load(model_widget.value, map_location='cuda')
except:
    checkpoint= torch.load(model_widget.value, map_location='cpu')
checkpoint['model_attributes']['out_features'] = len(original_db.columns)
checkpoint['model_attributes']['hidden_layers'] = [512, 256]
generator = GeneratorNet(**checkpoint['model_attributes'])
generator.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [35]:
os.makedirs("fake_data/" + original_db_name )

In [36]:
# size = original_db_size
# size = 200
size = 50
new_data = generator.create_data(size)
df = pd.DataFrame(new_data, columns=original_db.columns)
#Changes the name to be easier to read
# name = model_widget.value.split("/")[-1][10:-4] + "_size-" + str(size)
name = original_db_name + "_size-" + str(size)
df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)

In [84]:
#Do the same thing as the cells above but for all the files in the directory
import glob
for file in glob.glob(folder_name):
    name = file.split("/")[-1][10:-4]
    print(name)
    try:
        checkpoint= torch.load(file, map_location='cuda')
    except:
        checkpoint= torch.load(file, map_location='cpu')
    generator = GeneratorNet(**checkpoint['model_attributes'])
    generator.load_state_dict(checkpoint['model_state_dict'])
    size = original_db_size
    new_data = generator.create_data(size)
    df = pd.DataFrame(new_data, columns=original_db.columns)
    name = name + "_size-" + str(size)
    df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)

iabetes_10
